# Result Analysis

In [16]:
import json
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from os.path import join
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean
from scipy.stats import pearsonr

from img2vec_pytorch import Img2Vec
from PIL import Image

In [17]:
DATA_DIR = "../data"

In [18]:
def get_vecs_of_run(dir, img2vec_model):
    vec_dict = {}
    vec_dict['diff'] = json.load(open(join(dir, "difference_embeddings.json")))
    vec_dict['prompt'] = json.load(open(join(dir, "prompt_embeddings.json")))
    vec_dict['img'] = [img2vec_model.get_vec(Image.open(os.path.join(dir, f))) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) and f.endswith(".png")]

    return vec_dict


def get_all_runs_vecs(img2vec_model, data_dir=DATA_DIR, min_i=0):
    vecs = {}
    for run in os.listdir(data_dir):
        if os.path.isdir(join(data_dir, run)):
            imgs_count = len([f for f in os.listdir(join(data_dir, run)) if os.path.isfile(os.path.join(data_dir, run, f)) and f.endswith(".png")])
            if imgs_count >= min_i:
                vecs[run] = get_vecs_of_run(join(data_dir, run), img2vec_model)
    return vecs


def get_tsne(embeddings, labels=None, perplexity=30.0, random_state=20, title=''):
    fig = plt.figure()
    fig.suptitle(title)

    matrix = np.array(embeddings)
    tsne = TSNE(n_components=2, perplexity=perplexity, random_state=random_state, init='pca', learning_rate=200)
    vis_dims = tsne.fit_transform(matrix)
    x = vis_dims[:,0]
    y = vis_dims[:,1]
    weights = np.arange(1, len(embeddings) + 1)
    plt.scatter(x, y, c=weights, cmap="BuPu")
    if labels:
        [plt.text(i, j, l) for (i, j, l) in zip(x, y, labels)]
    plt.colorbar()

    return fig


def get_similarities(embeddings, labels=None, limit=False, title=''):
    fig = plt.figure()
    fig.suptitle(title)
    similarities = cosine_similarity(embeddings)[-1]
    x = np.arange(0, len(embeddings))
    y = similarities
    plt.scatter(x, similarities, marker='x')
    if labels:
        [plt.text(i, j, l) for (i, j, l) in zip(x, y, labels)]
    plt.xlabel("Iterations")
    plt.ylabel("Similarity")
    if limit:
        plt.ylim(0, 1)
    
    return fig


def get_euclidean_distances(embeddings, labels=None, title=''):
    fig = plt.figure()
    fig.suptitle(title)

    # calc euclidian distances of each embedding to the last one
    distances = [euclidean(embedding, embeddings[-1]) for embedding in embeddings]

    x = np.arange(len(embeddings))
    y = distances

    plt.scatter(x, y, marker='x')

    # optionally add labels
    if labels:
        [plt.text(i, j, l) for (i, j, l) in zip(x, y, labels)]

    plt.xlabel("Iterations")
    plt.ylabel("Euclidean Distance")

    return fig


def plot_embedding_magnitudes_with_correlation(img_embeddings, prompt_embeddings, labels=None, title=''):
    # clac magnitudes of embeds
    image_magnitudes = [np.linalg.norm(embedding) for embedding in img_embeddings]
    prompt_magnitudes = [np.linalg.norm(embedding) for embedding in prompt_embeddings]

    # calc pearson correlation between image and prompt magnitudes
    correlation_coefficient, _ = pearsonr(image_magnitudes, prompt_magnitudes)

    fig, ax = plt.subplots()
    title_with_correlation = f"{title} - Pearson Correlation: {correlation_coefficient:.2f}"
    fig.suptitle(title_with_correlation)

    x = np.arange(len(img_embeddings))
    ax.plot(x, image_magnitudes, marker='o', label='Image Embeddings')
    ax.plot(x, prompt_magnitudes, marker='x', label='Prompt Embeddings')

    if labels:
        for i, label in enumerate(labels):
            ax.text(x[i], max(image_magnitudes[i], prompt_magnitudes[i]), label, fontsize=8)

    ax.set_xlabel("Iterations")
    ax.set_ylabel("Magnitude")
    ax.legend()

    return fig


def embedding_instability(embeddings):
    distances = []
    for i in range(1, len(embeddings)):
        distance = euclidean(embeddings[i], embeddings[i-1])
        distances.append(distance)
    instability = np.std(distances)
    
    return instability


def normalized_embedding_instability(embeddings):
    # instability normalized by value range
    distances = [euclidean(embeddings[i], embeddings[i-1]) for i in range(1, len(embeddings))]
    instability = np.std(distances)
    value_range = np.max(embeddings) - np.min(embeddings)
    normalized_stability = instability / value_range if value_range != 0 else 0
    return normalized_stability


def get_cosine_similarity_boxplot(embeddings, title=''):
    fig = plt.figure(figsize=(8, 6))
    fig.suptitle(title)
    # ensure 2D numpy array
    embeddings = np.array(embeddings)
    
    # calc cosine similarity of each embedding to the last one
    last_embedding = embeddings[-1].reshape(1, -1)
    similarities = cosine_similarity(embeddings, last_embedding).flatten()
    
    plt.boxplot(similarities, vert=True, patch_artist=True)
    plt.ylabel('Cosine Similarity')
    plt.xticks([1], ['Embeddings'])
    #plt.ylim(0, 1)  # cosine similarity ranges from -1 to 1 -> -1 would indicate completely opposite
    plt.grid(True)
    
    return fig


def measure_constant_increase(embeddings):
    embeddings = np.array(embeddings)
    
    last_embedding = embeddings[-1].reshape(1, -1)
    similarities = cosine_similarity(embeddings, last_embedding).flatten()
    differences = np.diff(similarities)
    average_difference = np.mean(differences)

    return average_difference

In [19]:
# get img vectors using img2vec
img2vec = Img2Vec(cuda=False)

# get all vecs
vecs = get_all_runs_vecs(img2vec, min_i=7)

# print instability and constant increase for each run and each embedding type
for r in vecs:
    print(r)
    print("difference embeddings:")
    print(f"\tinstability: {embedding_instability(vecs[r]['diff'])}")
    print(f"\tnormalized instability: {normalized_embedding_instability(vecs[r]['diff'])}")
    print(f"\tconstant increase: {measure_constant_increase(vecs[r]['diff'])}")

    print("prompt embeddings:")
    print(f"\tinstability: {embedding_instability(vecs[r]['prompt'])}")
    print(f"\tnormalized instability: {normalized_embedding_instability(vecs[r]['prompt'])}")
    print(f"\tconstant increase: {measure_constant_increase(vecs[r]['prompt'])}")

    print("image embeddings:")
    print(f"\tinstability: {embedding_instability(vecs[r]['img'])}")
    print(f"\tnormalized instability: {normalized_embedding_instability(vecs[r]['img'])}")
    print(f"\tconstant increase: {measure_constant_increase(vecs[r]['img'])}")

    print("-"*20)



/home/linus/git/uni/PromptRefining/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/linus/git/uni/PromptRefining/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Blue_cone_on_10_gpt-4-vision-preview_dall-e-3_gpt-4-1106-preview_v0
difference embeddings:
	instability: 0.032666020060499595
	normalized instability: 0.03776692170852417
	constant increase: 0.008936695861809316
prompt embeddings:
	instability: 0.04822334688045794
	normalized instability: 0.05190098817894406
	constant increase: 0.009360986928447423
image embeddings:
	instability: 2.1406718290340208
	normalized instability: 0.24590945773131206
	constant increase: 0.02572503685951233
--------------------
a_city_stree_30_gpt-4-vision-preview_dall-e-3_gpt-4-1106-preview_v1
difference embeddings:
	instability: 0.0552075181060654
	normalized instability: 0.06182898676334064
	constant increase: 0.005929258776751023
prompt embeddings:
	instability: 0.056152636245958135
	normalized instability: 0.061286257711955554
	constant increase: 0.008702073540627901
image embeddings:
	instability: 3.015579488184652
	normalized instability: 0.3077282677803092
	constant increase: 0.01146627776324749
-------

Load data from the specified directory

Print data using T-SNE